<a href="https://colab.research.google.com/github/Krankile/ensemble-forecasting/blob/main/data_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Setup

In [1]:
!pip install kora -q
#from google.colab import drive
#drive.mount('/content/drive')
from kora import drive
drive.link_nbs()

In [2]:
import pickle
from ensemble import * 

importing Jupyter notebook from /nbs/ensemble.ipynb


After running Auto-encoding.ipynb, manual-feature-extraction.ipynb and ensemble.ipynb, we are left with a rather large amount of data. For instance, the pickle file of all the models in all the ensembles produced for the 5000 timesseries is of size 4.57G. To speed up the training processs in weight_net.ipynb, we can make a large dataset, of which all timesseries have an associated auto-encoding, manual-features and forecast for $n<9$ methods in the ensamble  

#Create the epic dataframe of DOOM

##Create a dictionary of dataframes 

First, we find the idx of the shuffled timeseries our ensamble fitted to, overnight, on our XX CPU

In [3]:
manual_features = pd.read_csv('/content/drive/MyDrive/Master, Ankile og Krange/Data/manual features/monthly_manual_features.csv')
manual_features.set_index("Unnamed: 0",inplace=True)

In [4]:
lst = [("auto_features",8),("meta_data",1),("Random_walk_with_drift",18),("Naive",18),("Snaive",18),("Theta",18),("Auto_Arima",18),("Eps",18),("STLM_AR",18),("actuals",18)] 
columns = ["id"] + manual_features.columns.tolist()
for tup in lst: 
  for i in range(tup[1]):
    columns.append(f"{tup[0]}_{i}")

epic_dataframe_of_doom = pd.DataFrame(columns=columns).set_index("id")

In [5]:
with open('/content/drive/MyDrive/Master, Ankile og Krange/Data/ensembles/ensembles_prot_4.pickle', 'rb') as handle:
    ensembles_5000 = pickle.load(handle)

In [6]:
for key in ensembles_5000:
  epic_dataframe_of_doom.loc[key] = 0


##Calculate $n$ forecast for each dataframe in the dictionary 

In [9]:
for idx, row in epic_dataframe_of_doom.iterrows():
  ensembles_5000[idx].models = ensembles_5000[idx].models[:3] + ensembles_5000[idx].models[4:]  #TODO: As of now, some Tbats models had problems regarding the seasonality of series. Remove and fix l8r
  forecasts = ensembles_5000[idx].forecast(18) #forecast horizon is 18 

  for forecast in forecasts:
    start_prediction_index = 0
    start_col = forecast[1].replace(" ","_") + "_0"
    end_col =  forecast[1].replace(" ","_") + "_17"
    epic_dataframe_of_doom.loc[idx, start_col: end_col] = forecast[0][start_prediction_index:]

del ensembles_5000 #delete ensembles_5000 from ram, as we have used it for everything we need it for

In [10]:
#show newly made rows 

##Add actual values fore forecasting accuracy testing at the end of the dataframe

In [11]:
actual_values = pd.read_csv('/content/drive/MyDrive/Master, Ankile og Krange/Data/train/Monthly-train.csv')
actual_values = actual_values.set_index("V1")


In [12]:
for idx, row in epic_dataframe_of_doom.iterrows():
  row = actual_values.loc[idx].dropna().to_list()[-18:] 
  epic_dataframe_of_doom.loc[idx, "actuals_0":"actuals_17"] = row 

##Calculate AutoEncoding add to each dataframe in the dictionary


##Calculate ManualFeatures and add to each dataframe in the dictionary

In [13]:
manual_features = pd.read_csv('/content/drive/MyDrive/Master, Ankile og Krange/Data/manual features/monthly_manual_features.csv')
manual_features.set_index("Unnamed: 0",inplace=True)
#manual_features

In [14]:
for idx, row in epic_dataframe_of_doom.iterrows():
  epic_dataframe_of_doom.loc[idx, "frequency":"seas_acf1"] = manual_features.iloc[int(idx[1:]),].to_list()

In [15]:
%%html
<marquee style='width: 30%; color: green;'><b>:D EPIC DATAFRAME OF DOOM :- ] EPIC DATAFRAME OF DOOM :) EPIC DATAFRAME OF DOOM</b></marquee>

In [16]:
epic_dataframe_of_doom

,frequency,nperiods,seasonal_period,trend,spike,linearity,curvature,e_acf1,e_acf10,seasonal_strength,peak,trough,entropy,x_acf1,x_acf10,diff1_acf1,diff1_acf10,diff2_acf1,diff2_acf10,seas_acf1,auto_features_0,auto_features_1,auto_features_2,auto_features_3,auto_features_4,auto_features_5,auto_features_6,auto_features_7,meta_data_0,Random_walk_with_drift_0,Random_walk_with_drift_1,Random_walk_with_drift_2,Random_walk_with_drift_3,Random_walk_with_drift_4,Random_walk_with_drift_5,Random_walk_with_drift_6,Random_walk_with_drift_7,Random_walk_with_drift_8,Random_walk_with_drift_9,Random_walk_with_drift_10,...,Eps_14,Eps_15,Eps_16,Eps_17,STLM_AR_0,STLM_AR_1,STLM_AR_2,STLM_AR_3,STLM_AR_4,STLM_AR_5,STLM_AR_6,STLM_AR_7,STLM_AR_8,STLM_AR_9,STLM_AR_10,STLM_AR_11,STLM_AR_12,STLM_AR_13,STLM_AR_14,STLM_AR_15,STLM_AR_16,STLM_AR_17,actuals_0,actuals_1,actuals_2,actuals_3,actuals_4,actuals_5,actuals_6,actuals_7,actuals_8,actuals_9,actuals_10,actuals_11,actuals_12,actuals_13,actuals_14,actuals_15,actuals_16,actuals_17
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
M38737,12,1,12,0.983646,7.73294e-08,8.14089,1.07307,0.601669,0.633872,0.285058,3,8,0.206258,0.925194,4.1324,0.173402,0.0866376,-0.411649,0.237882,0.303126,0,0,0,0,0,0,0,0,0,995.365,1005.49,1305.96,736.304,781.021,1253.69,1131.05,1612.85,1992.02,1867.59,2075.54,...,914.68,914.68,914.68,914.68,932.117,935.648,940.075,918.226,931.421,913.952,914.314,914.067,930.877,927.26,944.383,947.609,951.738,929.597,942.509,924.764,924.856,924.347,934.5,968.71,1040.7,1051.57,1061.47,1066.96,1080.36,1104.47,1115.69,1126.51,1152.04,1159.5,1159.27,1715.76,1739.43,1739.78,1796.99,1803.75
M12946,12,1,12,0.985464,4.83794e-09,14.4465,1.02306,0.548,0.443406,0.847091,12,2,0.373246,0.930258,6.87021,-0.0790797,0.401433,-0.332803,0.542903,0.780928,0,0,0,0,0,0,0,0,0,1571.57,2334.05,2153.58,1877.97,2556.82,2556.89,2007.72,1482.83,1235.31,1238.8,1567.91,...,1226,1226,1226,1226,1244.29,1249.66,1249.82,1254.99,1244.85,1241.31,1232.6,1230.94,1225.69,1232.22,1257.99,1259.04,1260,1268.53,1256.15,1253.94,1245.88,1240.69,1205,1215,1229,1234,1247,1258,1256,1263,1258,1254,1259,1265,1238,1248,1261,1264,1271,1281
M12038,12,1,12,0.945259,3.0647e-07,9.16749,3.60673,0.290871,0.200192,0.551336,9,1,0.476401,0.887472,4.2575,-0.269738,0.171321,-0.576454,0.482084,0.4612,0,0,0,0,0,0,0,0,0,1428.37,1938.94,2623.74,2937.47,2414.24,2272.57,2434.73,2830.73,3115.46,3752.41,2586.04,...,1294.62,1294.62,1294.62,1294.62,1297.31,1294.57,1284.8,1311.03,1292.24,1295.48,1298.27,1305.04,1295.71,1296.74,1300.52,1296.83,1286.86,1313.8,1296.22,1298.49,1300.97,1307.74,1310,1299,1336,1315,1336,1313,1325,1342,1320,1334,1306,1299,1335,1349,1371,1381,1377,1353
M7541,12,1,12,0.99648,9.98613e-10,8.33078,0.738756,0.917624,2.76072,0.0784861,9,2,0.378482,0.926266,4.30369,0.94842,4.86494,-0.166066,0.387607,0.273487,0,0,0,0,0,0,0,0,0,7392.51,2864.3,12565.3,12739.4,16490.6,17827.9,19679.1,18491.9,5689.66,-1394.91,-2158.31,...,5610.45,5610.45,5610.45,5610.45,5319.62,5221.12,5145.95,5113.02,5104.9,5134.23,5190.66,5270.66,5377.95,5511.91,5454.63,5595.07,5756.86,5953.35,6161.82,6391.8,6629.3,6868.22,5510,5400,5290,5170,5040,4900,4760,4610,4460,4320,4170,4030,3890,3760,3630,3510,3400,3290
M43477,12,1,12,0.99946,1.15566e-11,-14.8646,5.81061,0.771241,0.873208,0.160794,9,2,0.249708,0.996223,9.43443,0.638306,2.12093,-0.331534,0.147063,0.926214,0,0,0,0,0,0,0,0,0,5052.93,2035.21,2809.37,1083.74,6879.12,11346.8,7848.07,3156.33,2827.79,1438.62,3028.8,...,7106.1,7106.1,7106.1,7106.1,7261.08,7563.53,7537.74,7714.13,7668.32,6964.45,7010.18,7103.94,7262.11,7425.69,7570.65,7865.3,7832.09,8001.28,7948.46,7237.75,7276.81,7364.05,7138,7019,7028,7121,7113,7895,7947,8273,8166,8330,8099,8390,8327,8344,8373,8381,8025,8009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

##Write to file if wanted 

In [17]:
PATH = '/content/drive/MyDrive/Master, Ankile og Krange/Data/epic_dataframe_of_doom.csv'
epic_dataframe_of_doom.to_csv(PATH)